In [ ]:
import os
# These are the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables. 
aws_default_config = {
    #'AWS_NO_SIGN_REQUEST': 'YES', 
    'AWS_SECRET_ACCESS_KEY': 'fake',
    'AWS_ACCESS_KEY_ID': 'fake',
}

# To access public bucket, need to remove the AWS credentials in 
# the environment variables or the following error will occur.
# PermissionError: The AWS Access Key Id you provided does not exist in our records.

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [ ]:
import json
import logging

import click
import fsspec

from deafrica_conflux.cli.logs import logging_setup
from deafrica_conflux.io import check_dir_exists, check_file_exists, check_if_s3_uri
from deafrica_conflux.stack import stack_waterbodies_parquet_to_csv

In [ ]:
verbose = 1
# Path to the directory containing the parquet files output during polygon drill.
drill_output_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/drill_pq_files"
# Path to the directory write the csvs to
output_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/surface_area_change"
# JSON file containing the polygons ids.
polygon_ids_mapping_file = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/polygon_ids_mapping.json"

In [ ]:
# Set up logger.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [ ]:
# Support pathlib Paths
output_directory = str(output_directory)
drill_output_directory = str(drill_output_directory)
polygon_ids_mapping_file = str(polygon_ids_mapping_file)

In [ ]:
if not check_dir_exists(drill_output_directory):
    _log.error(f"Directory {drill_output_directory} does not exist!")
    raise FileNotFoundError(f"Directory {drill_output_directory} does not exist!)")

# Create the output directory if it does not exist.
if not check_dir_exists(output_directory):
    if check_if_s3_uri(output_directory):
        fsspec.filesystem("s3").makedirs(output_directory, exist_ok=True)
    else:
        fsspec.filesystem("file").makedirs(output_directory, exist_ok=True)
    _log.info(f"Created directory {output_directory}")

if not check_file_exists(polygon_ids_mapping_file):
    _log.error(f"File {polygon_ids_mapping_file} does not exist!")
    raise FileNotFoundError(f"File {polygon_ids_mapping_file} does not exist!)")

In [ ]:
# Read the polygons ids mapping file.
if check_if_s3_uri(polygon_ids_mapping_file):
    fs = fsspec.filesystem("s3")
else:
    fs = fsspec.filesystem("file")

with fs.open(polygon_ids_mapping_file) as f:
    polygon_ids_mapping = json.load(f)

# Find all the drill output parquet files
polygon_ids = list(polygon_ids_mapping.values())

In [ ]:
for polygon_id in polygon_ids:
    stack_waterbodies_parquet_to_csv(
        polygon_id=polygon_id,
        drill_output_directory=drill_output_directory,
        output_directory=output_directory,
    )